In [2]:
# from ipywidgets import interact
# from bokeh.io import push_notebook, show, output_notebook
# from bokeh.plotting import figure
import dill
import pandas as pd
import numpy as np
# import requests

# from ipywidgets import *
# from IPython.display import display
# from IPython.display import clear_output

# output_notebook()

In [ ]:
# body = pd.read_csv('ee_or_matched.csv',index_col=0, dtype=str)
# body_main = body[['or_name','or_name_matched','ee_name_matched','ticker','grant_doc_num','title', 'convey_text', 'exec_dt','cname']]
# body_main.head()
# titles = list(body_main['title'])

* client one: search on the key words of orphan drugs
* client two: want to know where other companies are heading to? are they forging a bunch of patents on some area?

*(micro seismic; passive seismic)*

In [3]:
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

In [4]:
# word1 is a list of words (separated strings)
def getSynonyms(word1):
        synonymList1 = []
        for data1 in word1:
            wordnetSynset1 = wn.synsets(data1)
            tempList1=[]
            for synset1 in wordnetSynset1:
                for synWords1 in synset1.lemma_names():
                     tempList1.append(synWords1)
            synonymList1.append(list(set(tempList1)))
        
        return synonymList1

In [5]:
stopwords_ls = stopwords.words('english')

In [ ]:
# key_titles=[]
# for t in tqdm(titles):
#     if str(t)!= 'nan':
#         filtered = [w for w in word_tokenize(t) if w.lower() not in stopwords_ls]
#         key_titles.append(filtered)
#     else:
#         key_titles.append(['nan'])

In [ ]:
# len(key_titles)
# body_main['title']=key_titles
# body_main.to_pickle('tokened_titles.pkd')

In [6]:
body = dill.load(open('tokened_titles.pkd', 'rb'))
body_ibm = body[body['ticker']=='IBM']

In [27]:
newbody = dill.load(open('ibm2000.pkd', 'rb'))
microsoft = dill.load(open('microsoft2000.pkd', 'rb'))

In [61]:
microsoft.head()

,assignees,inventors,patent_abstract,patent_date,citations,patent_number,patent_title,patent_type
0,Microsoft Corporation,"Steven J. Altschuler, Greg Ridgeway","Building resource (e.g., Internet content) and...",2000-01-04,5,6012052,Methods and apparatus for building resource tr...,utility
1,Microsoft Corporation,"Cory Alan Reina, Usama M. Fayyad, Paul S. Bradley",In one exemplary embodiment the invention prov...,2000-01-04,4,6012058,Scalable system for K-means clustering of larg...,utility
2,Microsoft Corporation,"Ronald A. Fein, Anthony D. Krueger",In an electronic word processing system enviro...,2000-01-04,6,6012075,Method and system for background grammar check...,utility
3,Microsoft Corporation,"Craig A. Link, Hoon Im",A method and system for determining network la...,2000-01-04,2,6012096,Method and system for peer-to-peer network lat...,utility
4,Microsoft Corporation,Bjarne Steensgaard,A terminating polymorphic type inference progr...,2000-01-11,4,6014518,Terminating polymorphic type inference program...,utility


In [62]:
sample = newbody[['assignees','patent_abstract']]
sample2 = microsoft[['assignees', 'patent_abstract']]

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import nltk

In [114]:
vectorizer = TfidfVectorizer(ngram_range=(5,5), stop_words='english')
# vectorizer2 = TfidfVectorizer(ngram_range=(4,4), stop_words='english')
target = newbody['patent_abstract']
# target2 = microsoft['patent_abstract']

tfidf = vectorizer.fit_transform(target)
# tfidf2 = vectorizer2.fit_transform(target2)
names = vectorizer.get_feature_names()
# names2 = vectorizer2.get_feature_names()
df = pd.DataFrame(tfidf.T.todense(), index=names, columns=range(len(target)))
# df2 = pd.DataFrame(tfidf2.T.todense(), index= names2, columns = range(len(target2)))

# K Means

In [115]:
k = 4
km_model = KMeans(n_clusters=k, init='k-means++', max_iter=50, n_init=2)
# km_model2 = KMeans(n_clusters=k, init='k-means++', max_iter=50, n_init=2)
km_model.fit(tfidf)
# km_model2.fit(tfidf2)

order_centroids = km_model.cluster_centers_.argsort()[:, ::-1]
# order_centroids2 = km_model2.cluster_centers_.argsort()[:, ::-1]

In [116]:
cluster = dict()
# cluster2 = dict()
for i in range(k):
    cluster[i] = []
#     cluster2[i] = []
    for ind in order_centroids[i, :15]:
        cluster[i].append(names[ind])
#     for ind in order_centroids2[i, :15]:
#         cluster2[i].append(names2[ind])
        
centroids = pd.DataFrame(km_model.cluster_centers_).T
# centroids2 = pd.DataFrame(km_model2.cluster_centers_).T
centroids.index = names
# centroids2.index = names2

In [117]:
n=0
m=10
df1 = centroids[0].to_frame().sort_values(by=0, ascending=False)[n:m].reset_index()
df2 = centroids[1].to_frame().sort_values(by=1, ascending=False)[n:m].reset_index()
df3 = centroids[2].to_frame().sort_values(by=2, ascending=False)[n:m].reset_index()
df4 = centroids[3].to_frame().sort_values(by=3, ascending=False)[n:m].reset_index()

# df21 = centroids2[0].to_frame().sort_values(by=0, ascending=False)[n:m].reset_index()
# df22 = centroids2[1].to_frame().sort_values(by=1, ascending=False)[n:m].reset_index()
# df23 = centroids2[2].to_frame().sort_values(by=2, ascending=False)[n:m].reset_index()
# df24 = centroids2[3].to_frame().sort_values(by=3, ascending=False)[n:m].reset_index()

In [118]:
cluster_df = pd.concat([df1, df2, df3, df4], axis=1)
# cluster_df2 = pd.concat([df21, df22, df23, df24], axis=1)
# cluster_df

In [119]:
cluster_df

,index,0,index,1,index,2,index,3
0,gate formed portion channel region,0.147442,direct access storage device dasd,0.000521,functions cache controller layered provided,0.097046,card assembly distribution electrical signals,0.104181
1,substantially electrically isolated floating gate,0.147442,methods systems computer program products,0.000461,cache controller layered provided generic,0.097046,circuit card assembly distribution electrical,0.104181
2,source region recessed semiconductor substrate,0.147442,apparatus article manufacture computer impleme...,0.000349,controller layered provided generic interfaces,0.097046,backplane circuit card assembly distribution,0.104181
3,gate formed entire channel region,0.147442,method apparatus article manufacture computer,0.000332,architectural operations initiated local proce...,0.096406,contains backplane circuit card assembly,0.104181
4,region insulation layer conductive control,0.147442,systems computer program products provided,0.000316,cache architectural functions cache controller,0.088277,server having concurrent maintenance capability,0.065906
5,source region drain region separated,0.147442,data storage device connected computer,0.000280,architectural functions cache controller layered,0.088277,embodiment intelligent redundant array indepen...,0.065906
6,region conductive floating gate formed,0.147442,present invention provides method apparatus,0.000262,handles certain type operation easily,0.057394,intelligent redundant array independent disks,0.065906
7,substrate separated portion channel region,0.147442,composition containing fluoride containing com...,0.000260,autonomous functional units functional unit,0.057394,electronic contains backplane circuit card,0.065906
8,entire channel region method fabricating,0.147442,method computer program product provided,0.000260,divided distinct largely autonomous functional,0.057394,redundant array independent disks raid,0.065906
9,layer conductive control gate formed,0.147442,programmable resources field programmable gate,0.000239,specific functions cache controller layered,0.057394,raid storage server having concurrent,0.065906


In [102]:
cluster_df2

,index,0,index,1,index,2,index,3
0,present invention provides method,0.000801,methods systems computer program,0.000551,delay execution second class,0.117885,progress message exchange pattern,0.180688
1,handlers priority use queues,0.000659,systems computer program products,0.000550,class services provided network,0.117885,message exchange pattern definition,0.162064
2,ensuring smooth flow electronic,0.000659,method computer readable medium,0.000541,used trigger throttling response,0.117885,state message exchange pattern,0.161103
3,dynamic rendering prior steps,0.000659,invention pointed appending claims,0.000514,services execution concurrently delay,0.117885,definition allows valid messages,0.108043
4,dynamic rendering priority event,0.000659,scope present invention pointed,0.000514,services provided network server,0.117885,pattern definition allows valid,0.108043
5,rendering priority event handlers,0.000659,present invention pointed appending,0.000514,varied include additional levels,0.117885,exchange pattern definition allows,0.108043
6,order events occur performing,0.000659,scope patent scope present,0.000462,levels bandwidth used trigger,0.117885,code generated state simulates,0.108043
7,process ensuring smooth flow,0.000659,limit scope patent scope,0.000462,response include subsets class,0.117885,computing systems communicate order,0.060229
8,queues order events occur,0.000659,abstract intended limit scope,0.000462,class services implementation throttling,0.117885,tree node represents state,0.060229
9,performing dynamic rendering prior,0.000659,intended limit scope patent,0.000462,throttling response blocks class,0.117885,receipt certain messages cause,0.060229


# Affinity Propagation

In [103]:
from sklearn.cluster import AffinityPropagation

In [112]:
model = AffinityPropagation(preference=0.1)
model.fit(tfidf)
model.cluster_centers_indices_